In [ ]:
import pandas as pd
import numpy as np
import gzip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Clinvar variants treatment 

- Pathogenic, likely pathogenic and conflicting variants selection

In [ ]:
with gzip.open('/content/drive/MyDrive/GRAR/Paper_Surdez/clinvar.vcf.gz','rt') as file:
  count=0
  for line in file:
    if line.startswith('#'):
      count+=1
    else:
      break
print(count)

df_clinvar = pd.read_table('/content/drive/MyDrive/GRAR/Paper_Surdez/clinvar.vcf.gz', header=None,skiprows=count)
df_clinvar_total = df_clinvar.copy()
df_clinvar

28


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,0,1,2,3,4,5,6,7
0,1,925952,1019397,G,A,.,.,ALLELEID=1003021;CLNDISDB=MedGen:CN517202;CLND...
1,1,930139,1125147,C,T,.,.,ALLELEID=1110865;CLNDISDB=MedGen:CN517202;CLND...
2,1,930165,1164676,G,A,.,.,ALLELEID=1153701;CLNDISDB=MedGen:CN517202;CLND...
3,1,930187,1144630,C,T,.,.,ALLELEID=1131738;CLNDISDB=MedGen:CN517202;CLND...
4,1,930188,846933,G,A,.,.,ALLELEID=824438;CLNDISDB=MedGen:CN517202;CLNDN...
...,...,...,...,...,...,...,...,...
1083268,MT,16013,690274,A,G,.,.,"ALLELEID=677666;CLNDISDB=MONDO:MONDO:0010789,M..."
1083269,MT,16015,690276,T,TATTCTCTGTTCTTTC,.,.,"ALLELEID=677667;CLNDISDB=MONDO:MONDO:0010789,M..."
1083270,MT,16017,690275,T,C,.,.,"ALLELEID=677668;CLNDISDB=MONDO:MONDO:0010789,M..."
1083271,MT,16021,690277,C,T,.,.,"ALLELEID=677669;CLNDISDB=MONDO:MONDO:0010789,M..."


In [ ]:
df_clinvar = df_clinvar[df_clinvar[7].str.contains('CLNSIG=Pathogenic|CLNSIG=Likely_pathogenic|CLNSIG=Conflicting_interpretations_of_pathogenicity')]
regions_regex = r'.*CLNVC=(.*?);.*MC=SO.*?\|(.*?)[\,|\;].*'  
df_clinvar[['var_type','mol_consequence']] = df_clinvar[7].str.extract(regions_regex,expand=True)
df_clinvar = df_clinvar[[0,1,3,4,'var_type','mol_consequence']]
df_clinvar.columns = ['chr','pos','ref','alt','var_type','mol_consequence']
df_clinvar.head(20)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,chr,pos,ref,alt,var_type,mol_consequence
322,1,943995,C,T,single_nucleotide_variant,nonsense
376,1,964512,C,A,single_nucleotide_variant,missense_variant
391,1,1013983,G,A,single_nucleotide_variant,splice_acceptor_variant
414,1,1014143,C,T,single_nucleotide_variant,nonsense
435,1,1014316,C,CG,Duplication,frameshift_variant
444,1,1014359,G,T,single_nucleotide_variant,nonsense
478,1,1020239,G,C,single_nucleotide_variant,missense_variant
496,1,1022225,G,A,single_nucleotide_variant,missense_variant
504,1,1022313,A,T,single_nucleotide_variant,missense_variant
540,1,1040679,C,T,single_nucleotide_variant,synonymous_variant


In [ ]:
df_clinvar['end'] = np.where( (df_clinvar['ref'].str.len() > df_clinvar['alt'].str.len() ) , df_clinvar['pos'].astype(int)+1, df_clinvar['pos'].astype(int)+ df_clinvar['alt'].str.len())
df_clinvar = df_clinvar[['chr', 'pos', 'end', 'ref', 'alt','var_type','mol_consequence']]
df_clinvar.columns = ['Chromosome','Start','End','Ref','Alt','var_type','mol_consequence']
df_clinvar.head()

,Chromosome,Start,End,Ref,Alt,var_type,mol_consequence
322,1,943995,943996,C,T,single_nucleotide_variant,nonsense
376,1,964512,964513,C,A,single_nucleotide_variant,missense_variant
391,1,1013983,1013984,G,A,single_nucleotide_variant,splice_acceptor_variant
414,1,1014143,1014144,C,T,single_nucleotide_variant,nonsense
435,1,1014316,1014318,C,CG,Duplication,frameshift_variant


In [ ]:
df_clinvar[df_clinvar.Chromosome == 'M']

,Chromosome,Start,End,Ref,Alt,var_type,mol_consequence


### Opening deaf curated genes

In [ ]:
df_genes = pd.read_table('/content/drive/MyDrive/GRAR/Paper_Surdez/Genes_surdez.csv', sep=";", usecols=['Genes'])
df_genes = df_genes.sort_values('Genes').drop_duplicates()
print(f'Unique genes on the list -- {len(df_genes)}')

Unique genes on the list -- 128


### Opening MANE and RefSelect transcripts position
- Comparing the transcripts with the genes list

**results:**
- 3 genes weren't found: MIR96, DFNX3, KCNJ10

In [ ]:
df_MANEbed = pd.read_table('/content/drive/MyDrive/GRAR/Paper_Surdez/canonical_transcripts_MANE7_sorted.bed',header=None)
df_MANEbed.head(3)

,0,1,2,3,4,5
0,chr1,664,1237,PRAMEF22__NM_001100631.2_1__exon__1__chr1__f,0,+
1,chr1,65419,65433,OR4F5__NM_001005484.2__exon__1__chr1__f,0,+
2,chr1,65565,65573,OR4F5__NM_001005484.2__exon__2__chr1__f,0,+


In [ ]:
regions_regex = r'(.*)__(N.*)__(exon__[0-9]+)_'
df_MANEbed[['gene', 'transcript', 'exon']] = df_MANEbed[3].str.extract(regions_regex, expand=True)
df_MANEbed.drop(columns=[3,4,5], inplace = True)
df_MANEbed.head(3)

,0,1,2,gene,transcript,exon
0,chr1,664,1237,PRAMEF22,NM_001100631.2_1,exon__1
1,chr1,65419,65433,OR4F5,NM_001005484.2,exon__1
2,chr1,65565,65573,OR4F5,NM_001005484.2,exon__2


In [ ]:
df_merged = pd.merge(df_MANEbed, df_genes, left_on=['gene'], right_on=['Genes'], how='right', indicator= True)
df_merged[df_merged['_merge'] == 'right_only']

,0,1,2,gene,transcript,exon,Genes,_merge
483,NaN,NaN,NaN,NaN,NaN,NaN,DFNX3,right_only
857,NaN,NaN,NaN,NaN,NaN,NaN,KCNJ10,right_only
989,NaN,NaN,NaN,NaN,NaN,NaN,MIR96,right_only


### Opening RefSeq transcripts from Varstation Database
- Comparing the transcripts with the genes list

**results:**
- 2 genes weren't found: DFNX3, KCNJ10 </br>
- Assuming this dataframe as main
- We are considering all possible transcripts positions



In [ ]:
df_transcript = pd.read_table('/content/drive/MyDrive/GRAR/Paper_Surdez/refSeq_transcript_regions_109.20210514.hg38.bed', header=None)
df_transcript.head(3)

,0,1,2,3,4,5,6
0,chr1,11873,14409,DDX11L1,NR_046018.2,0,+
1,chr1,14361,29370,WASH7P,NR_024540.1,0,-
2,chr1,17368,17436,MIR6859-1,NR_106918.1,0,-


In [ ]:
df_transcritos_merged = pd.merge(df_transcript, df_genes, left_on=[3], right_on=['Genes'], how='right', indicator= True)
df_transcritos_merged[df_transcritos_merged['_merge'] == 'right_only']

,0,1,2,3,4,5,6,Genes,_merge
85,NaN,NaN,NaN,DFNX3,NaN,NaN,NaN,DFNX3,right_only
188,NaN,NaN,NaN,KCNJ10,NaN,NaN,NaN,KCNJ10,right_only


In [ ]:
df_transcritos_merged = df_transcritos_merged[df_transcritos_merged['_merge'] == 'both']
df_transcritos_merged = df_transcritos_merged.iloc[:,:5]

In [ ]:
df_transcritos_merged[0] = df_transcritos_merged[0].str.replace('chr','')
df_transcritos_merged[[1,2]] = df_transcritos_merged[[1,2]].astype(int) #.str.replace('chr','')
df_transcritos_merged.head()


,0,1,2,3,4
0,16,15949751,16141281,ABCC1,NM_004996.4
1,17,81509970,81512799,ACTG1,NR_037688.3
2,17,81510689,81512354,ACTG1,NM_001199954.3
3,17,81510689,81512354,ACTG1,NM_001614.5
4,7,45574543,45713995,ADCY1,NM_021116.4


In [ ]:
df_transcritos_merged.columns = ['Chromosome','Start','End','Gene','Transcript']
df_transcritos_merged.head(3)

,Chromosome,Start,End,Gene,Transcript
0,16,15949751,16141281,ABCC1,NM_004996.4
1,17,81509970,81512799,ACTG1,NR_037688.3
2,17,81510689,81512354,ACTG1,NM_001199954.3


In [ ]:
df_transcritos_merged = df_transcritos_merged.groupby('Gene').agg({'Chromosome':'first','Start':'min', 'End':'max',}).reset_index()
df_transcritos_merged.sort_values(['Chromosome','Start'], inplace=True)
df_transcritos_merged.head(3)

,Gene,Chromosome,Start,End
31,ESPN,1,6424955,6460146
40,GJB3,1,34784762,34785575
54,KCNQ4,1,40784093,40838523


In [ ]:
#df_transcritos_merged.to_csv('/content/drive/MyDrive/GRAR/Paper_Surdez/transcripts_surdez.txt', index = None, header = False, sep = '\t')

In [ ]:
df_transcritos_merged[df_transcritos_merged.Chromosome == 'MT']

,Gene,Chromosome,Start,End


### Selecting Clinvar variants that fulfill the transcript positions

**results:**
- It's 5420 Pathogenic/Likely Pathogenic/Conflicting variants on deaf selected genes 

In [ ]:
!pip install pyranges
import pyranges as pr

In [ ]:
transcritos_pr = pr.PyRanges(df_transcritos_merged)
clinvar_pr = pr.PyRanges(df_clinvar)

In [ ]:
# Filtering and annotating clinvar variants 
annotate_transcript = clinvar_pr.join(transcritos_pr).drop(like="_b")
annotate_transcript

,Chromosome,Start,End,Ref,Alt,var_type,mol_consequence,Gene
0,1,6425219,6425220,G,A,single_nucleotide_variant,nonsense,ESPN
1,1,6425247,6425248,C,T,single_nucleotide_variant,nonsense,ESPN
2,1,6445936,6445937,G,A,single_nucleotide_variant,splice_donor_variant,ESPN
3,1,6448931,6448933,C,CG,Duplication,frameshift_variant,ESPN
4,1,6451602,6451603,G,C,single_nucleotide_variant,splice_acceptor_variant,ESPN
...,...,...,...,...,...,...,...,...
5415,X,130147792,130147793,G,A,single_nucleotide_variant,missense_variant,AIFM1
5416,X,130147804,130147805,G,A,single_nucleotide_variant,missense_variant,AIFM1
5417,X,130149478,130149479,C,T,single_nucleotide_variant,missense_variant,AIFM1
5418,X,130156576,130156577,G,C,single_nucleotide_variant,missense_variant,AIFM1


### Dealing with mitocondrial variants

**results:**
- From 120 variants only 22 were found at clinvar database 


In [ ]:
## Dataframe transcrito mitocondrial 
df_mitocond = df_transcript[df_transcript[0] == "chrM"].copy()
df_mitocond.head()

,0,1,2,3,4,5,6
77601,chrM,576,647,MT-TF,rna-TRNF,0,+
77602,chrM,647,1601,MT-RNR1,rna-RNR1,0,+
77603,chrM,1601,1670,MT-TV,rna-TRNV,0,+
77604,chrM,1670,3229,MT-RNR2,rna-RNR2,0,+
77605,chrM,3229,3304,MT-TL1,rna-TRNL1,0,+


In [ ]:
df_mt = pd.read_table('/content/drive/MyDrive/GRAR/Paper_Surdez/MT_variants.txt',)
df_mt['Position'] = df_mt['Position'].str.replace('1592C>6T','15926C>T') #correcting human issue
regions_regex = (r'([0-9]+)([A-Z]+)>([A-Z]+)')

df_mt[['Start','Ref','Alt']] = df_mt['Position'].str.extract(regions_regex,expand=True)


In [ ]:
df_mt.loc[(df_mt.Position.str.contains("del|dup")), ['Start']] = df_mt.Position.str.extract(r'([0-9]+)[del|dup].*')[0]
df_mt.loc[(df_mt.Position.str.contains("del|dup")), ['Ref']] = df_mt.Position.str.extract(r'.*[del|dup](.*)')[0]
df_mt.loc[(df_mt.Position.str.contains("del")), ['Alt']] = '.'
df_mt.loc[(df_mt.Position.str.contains("dup")), ['Alt']] = df_mt.Position.str.extract(r'.*dup(.*)')[0]*2
df_mt= df_mt[['Gene','Position', 'Start','Ref','Alt']]



In [ ]:
df_mt['End'] = np.where( (df_mt['Ref'].str.len() > df_mt['Alt'].str.len() ) , df_mt['Start'].astype(int)+1, df_mt['Start'].astype(int)+ df_mt['Alt'].str.len())
df_mt['Chromosome'] = 'M'
df_mt[['Start','End']] = df_mt[['Start','End']].astype(int)
df_mt

,Gene,Position,Start,Ref,Alt,End,Chromosome
0,MT-RNR1,1517A>C,1517,A,C,1518,M
1,MT-RNR1,1537C>T,1537,C,T,1538,M
2,MT-RNR1,801A>G,801,A,G,802,M
3,MT-TG,10003T>C,10003,T,C,10004,M
4,MT-TG,10005A>G,10005,A,G,10006,M
...,...,...,...,...,...,...,...
115,MT-RNR1,960dupC,960,C,CC,962,M
116,MT-RNR1,988G>A,988,G,A,989,M
117,MT-RNR1,990T>C,990,T,C,991,M
118,MT-TN,5715A>G,5715,A,G,5716,M


In [ ]:
## Checking overlap with clinar database
df_clinvar_merge = pd.merge(df_clinvar, df_mt, left_on=['Chromosome','Start','End', 'Ref', 'Alt'], right_on=['Chromosome','Start','End', 'Ref', 'Alt'],)
df_clinvar_merge

,Chromosome,Start,End,Ref,Alt,var_type,mol_consequence,Gene,Position


### Merging variants - Genes and MT

**results:**
- Output file to use on filter script 


In [ ]:
annotate_transcript_df = annotate_transcript.df
annotate_transcript_df = annotate_transcript_df[['Chromosome', 'Start']]
annotate_transcript_df

,Chromosome,Start
0,1,6425219
1,1,6425247
2,1,6445936
3,1,6448931
4,1,6451602
...,...,...
5415,X,130147792
5416,X,130147804
5417,X,130149478
5418,X,130156576


In [ ]:
df_mt.head()
df_mt = df_mt[['Chromosome','Start']]
df_mt

,Chromosome,Start
0,M,1517
1,M,1537
2,M,801
3,M,10003
4,M,10005
...,...,...
115,M,960
116,M,988
117,M,990
118,M,5715


In [ ]:
df_final = pd.concat([annotate_transcript_df, df_mt])
df_final = df_final.drop_duplicates()
df_final['Chromosome'] = df_final['Chromosome'].map('chr{}'.format)
df_final
#df_final.to_csv('/content/drive/MyDrive/GRAR/Paper_Surdez/All_variants_position.txt', index = None, header = False, sep = '\t')

,Chromosome,Start
0,chr1,6425219
1,chr1,6425247
2,chr1,6445936
3,chr1,6448931
4,chr1,6451602
...,...,...
114,chrM,960
116,chrM,988
117,chrM,990
118,chrM,5715


### Metrics to be considered between all filtered variants

**results**
- Quantity of each variant type
- Quantity of molecular consequences within var type
- Final variants - not includins genes FAM189A2, DFNX3, KCNJ10

In [ ]:
df_final = annotate_transcript.df
df_final_genes = df_final['Gene'].drop_duplicates().copy()

genes_diff = pd.merge(df_genes, df_final_genes, left_on='Genes', right_on='Gene', how='outer', indicator=True )
genes_diff[genes_diff['_merge'] == 'left_only']

,Genes,Gene,_merge
24,DFNX3,NaN,left_only
36,FAM189A2,NaN,left_only
55,KCNJ10,NaN,left_only


In [ ]:

df_final.groupby(by=['var_type']).size().reset_index()

,var_type,0
0,Deletion,612
1,Duplication,225
2,Indel,42
3,Insertion,31
4,Inversion,1
5,Microsatellite,105
6,single_nucleotide_variant,4359


In [ ]:
df_final.groupby(by=['var_type','mol_consequence']).size()

var_type                   mol_consequence                    
Deletion                   frameshift_variant                      504
                           genic_downstream_transcript_variant       2
                           genic_upstream_transcript_variant         1
                           inframe_deletion                         21
                           inframe_indel                             2
                           initiatior_codon_variant                  1
                           intron_variant                           14
                           non-coding_transcript_variant             5
                           nonsense                                 22
                           splice_acceptor_variant                  17
                           splice_donor_variant                     22
                           stop_lost                                 1
Duplication                frameshift_variant                      186
              

### Annotating variants after merge with clinvar

In [ ]:
df_after_merge = pd.read_table('/content/drive/MyDrive/GRAR/Paper_Surdez/gene_names_alleles.txt', sep ='\t')
df_after_merge['chr'] = df_after_merge['chr'].str.replace('chr','')
#df_after_merge['chr'] = df_after_merge['chr'].str.replace('MT','M')
df_after_merge

,chr,position,gene_names,ref,alt,alt_list,var_type,mol_consequence,biallelic
0,1,6425219,ESPN,G,A,NaN,single_nucleotide_variant,nonsense,yes
1,1,6425247,ESPN,C,T,NaN,single_nucleotide_variant,nonsense,yes
2,1,6445936,ESPN,G,A,NaN,single_nucleotide_variant,splice_donor_variant,yes
3,1,6448931,ESPN,C,CG,NaN,Duplication,frameshift_variant,yes
4,1,6451602,ESPN,G,C,NaN,single_nucleotide_variant,splice_acceptor_variant,yes
...,...,...,...,...,...,...,...,...,...
5325,MT,15927,MT-TT,G,A,NaN,single_nucleotide_variant,NaN,yes
5326,MT,15975,MT-TP,C,T,NaN,single_nucleotide_variant,NaN,yes
5327,MT,15992,MT-TP,A,G,NaN,single_nucleotide_variant,NaN,yes
5328,MT,15997,MT-TP,T,C,NaN,single_nucleotide_variant,NaN,yes


In [ ]:
clinsig_regex = r'.*CLNSIG=(.*?);.*' 
#CLNSIG=Pathogenic|CLNSIG=Likely_pathogenic|CLNSIG=Conflicting_interpretations_of_pathogenicity 
df_clinvar_total['pathogenicity'] = df_clinvar_total[7].str.extract(clinsig_regex, expand=True)
df_clinvar_total = df_clinvar_total[[0,1,3,4,'pathogenicity']]
df_clinvar_total

,0,1,3,4,pathogenicity
0,1,925952,G,A,Uncertain_significance
1,1,930139,C,T,Likely_benign
2,1,930165,G,A,Benign
3,1,930187,C,T,Likely_benign
4,1,930188,G,A,Uncertain_significance
...,...,...,...,...,...
1083268,MT,16013,A,G,Uncertain_significance
1083269,MT,16015,T,TATTCTCTGTTCTTTC,Uncertain_significance
1083270,MT,16017,T,C,Benign
1083271,MT,16021,C,T,Uncertain_significance


In [ ]:
df_clinvar_total[df_clinvar_total['pathogenicity'].isnull()]

,0,1,3,4,pathogenicity
6481,1,7984971,G,A,NaN
7132,1,9975751,G,A,NaN
17540,1,25284610,G,T,NaN
17545,1,25290715,C,T,NaN
17547,1,25290760,A,C,NaN
...,...,...,...,...,...
1076266,X,154030761,CTGCTGCGCCCCTTGGGGCTGCTCTCCT,C,NaN
1076287,X,154030791,CTTTTCCGCCCAGGGCTCTTACAGGTCT,C,NaN
1076313,X,154030839,TCA,T,NaN
1079407,X,154534438,G,A,NaN


In [ ]:
df_clinvar_total.columns = ['chr','position','ref','alt','pathogenicity']
df_clinvar_total

,chr,position,ref,alt,pathogenicity
0,1,925952,G,A,Uncertain_significance
1,1,930139,C,T,Likely_benign
2,1,930165,G,A,Benign
3,1,930187,C,T,Likely_benign
4,1,930188,G,A,Uncertain_significance
...,...,...,...,...,...
1083268,MT,16013,A,G,Uncertain_significance
1083269,MT,16015,T,TATTCTCTGTTCTTTC,Uncertain_significance
1083270,MT,16017,T,C,Benign
1083271,MT,16021,C,T,Uncertain_significance


In [ ]:
df_after_merge_annotate = pd.merge(df_after_merge, df_clinvar_total, left_on=['chr','position','ref', 'alt'], right_on=['chr','position','ref', 'alt'], how='left' )
df_after_merge_annotate

,chr,position,gene_names,ref,alt,alt_list,var_type,mol_consequence,biallelic,pathogenicity
0,1,6425219,ESPN,G,A,NaN,single_nucleotide_variant,nonsense,yes,NaN
1,1,6425247,ESPN,C,T,NaN,single_nucleotide_variant,nonsense,yes,NaN
2,1,6445936,ESPN,G,A,NaN,single_nucleotide_variant,splice_donor_variant,yes,NaN
3,1,6448931,ESPN,C,CG,NaN,Duplication,frameshift_variant,yes,NaN
4,1,6451602,ESPN,G,C,NaN,single_nucleotide_variant,splice_acceptor_variant,yes,NaN
...,...,...,...,...,...,...,...,...,...,...
5325,MT,15927,MT-TT,G,A,NaN,single_nucleotide_variant,NaN,yes,Benign
5326,MT,15975,MT-TP,C,T,NaN,single_nucleotide_variant,NaN,yes,NaN
5327,MT,15992,MT-TP,A,G,NaN,single_nucleotide_variant,NaN,yes,NaN
5328,MT,15997,MT-TP,T,C,NaN,single_nucleotide_variant,NaN,yes,NaN


In [ ]:
df_after_merge_annotate.to_csv('/content/drive/MyDrive/GRAR/Paper_Surdez/Variants_after_merge_annotated.txt', index = None, header = True, sep = '\t')

In [ ]:
df_after_merge_annotate[~df_after_merge_annotate['pathogenicity'].isnull()]

,chr,position,gene_names,ref,alt,alt_list,var_type,mol_consequence,biallelic,pathogenicity
4847,20,62260083,OSBPL2,CTG,C,NaN,Deletion,frameshift_variant,yes,Pathogenic
4848,20,62260095,OSBPL2,CCT,C,NaN,Microsatellite,frameshift_variant,yes,Pathogenic
4849,20,62260100,OSBPL2,CAA,C,NaN,Deletion,frameshift_variant,yes,Pathogenic
4850,20,62260119,OSBPL2,AAC,A,NaN,Microsatellite,frameshift_variant,yes,Pathogenic
4851,21,36461002,CLDN14,C,T,NaN,single_nucleotide_variant,missense_variant,yes,Pathogenic/Likely_pathogenic
...,...,...,...,...,...,...,...,...,...,...
5319,MT,14693,MT-TE,A,G,NaN,single_nucleotide_variant,NaN,yes,Benign
5321,MT,15902,MT-TT,A,G,NaN,single_nucleotide_variant,NaN,yes,Benign
5322,MT,15908,MT-TT,T,C,NaN,single_nucleotide_variant,NaN,yes,Benign
5324,MT,15926,MT-TT,C,T,NaN,single_nucleotide_variant,NaN,yes,Benign


In [ ]:
df_variants_summary = pd.read_table('/content/drive/MyDrive/GRAR/Paper_Surdez/variant_summary.txt', sep ='\t')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [ ]:
df_variants_summary = df_variants_summary[df_variants_summary['Assembly'] == 'GRCh38']
df_variants_summary

,#AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,ClinSigSimple,LastEvaluated,RS# (dbSNP),nsv/esv (dbVar),RCVaccession,PhenotypeIDS,PhenotypeList,Origin,OriginSimple,Assembly,ChromosomeAccession,Chromosome,Start,Stop,ReferenceAllele,AlternateAllele,Cytogenetic,ReviewStatus,NumberSubmitters,Guidelines,TestedInGTR,OtherIDs,SubmitterCategories,VariationID,PositionVCF,ReferenceAlleleVCF,AlternateAlleleVCF
1,15041,Indel,NM_014855.3(AP5Z1):c.80_83delinsTGCTGTAAACTGTA...,9907,AP5Z1,HGNC:22197,Pathogenic,1,-,397704705,-,RCV000000012,"MONDO:MONDO:0013342,MedGen:C3150901,OMIM:61364...","Spastic paraplegia 48, autosomal recessive",germline;unknown,germline,GRCh38,NC_000007.14,7,4781213,4781216,na,na,7p22.1,"criteria provided, single submitter",2,-,N,"ClinGen:CA215070,OMIM:613653.0001",3,2,4781213,GGAT,TGCTGTAAACTGTAACTGTAAA
3,15042,Deletion,NM_014855.3(AP5Z1):c.1413_1426del (p.Leu473fs),9907,AP5Z1,HGNC:22197,Pathogenic,1,"Jun 29, 2010",397704709,-,RCV000000013,"MONDO:MONDO:0013342,MedGen:C3150901,OMIM:61364...","Spastic paraplegia 48, autosomal recessive",germline,germline,GRCh38,NC_000007.14,7,4787730,4787743,na,na,7p22.1,no assertion criteria provided,1,-,N,"ClinGen:CA215072,OMIM:613653.0002",1,3,4787729,GCTGCTGGACCTGCC,G
5,15043,single nucleotide variant,NM_014630.3(ZNF592):c.3136G>A (p.Gly1046Arg),9640,ZNF592,HGNC:28986,Uncertain significance,0,"Jun 29, 2015",150829393,-,RCV000000014,"MONDO:MONDO:0033005,MedGen:C4551772,OMIM:25130...",Galloway-Mowat syndrome 1,germline,germline,GRCh38,NC_000015.10,15,84799209,84799209,na,na,15q25.3,no assertion criteria provided,1,-,N,"ClinGen:CA210674,UniProtKB:Q92610#VAR_064583,O...",1,4,84799209,G,A
7,15044,single nucleotide variant,NM_017547.4(FOXRED1):c.694C>T (p.Gln232Ter),55572,FOXRED1,HGNC:26927,Pathogenic,1,"Dec 30, 2019",267606829,-,RCV000578659|RCV001194045|RCV000000015,"MedGen:CN517202|MONDO:MONDO:0009723,MedGen:C00...",not provided|Leigh syndrome|Mitochondrial comp...,germline,germline,GRCh38,NC_000011.10,11,126275389,126275389,na,na,11q24.2,"criteria provided, multiple submitters, no con...",3,-,N,"ClinGen:CA113792,OMIM:613622.0001",3,5,126275389,C,T
9,15045,single nucleotide variant,NM_017547.4(FOXRED1):c.1289A>G (p.Asn430Ser),55572,FOXRED1,HGNC:26927,Pathogenic,1,"Oct 01, 2010",267606830,-,RCV000000016,"MONDO:MONDO:0032624,MedGen:C4748791,OMIM:618241","Mitochondrial complex 1 deficiency, nuclear ty...",germline,germline,GRCh38,NC_000011.10,11,126277517,126277517,na,na,11q24.2,no assertion criteria provided,1,-,N,"UniProtKB:Q96CU9#VAR_064571,OMIM:613622.0002,C...",1,6,126277517,A,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2262902,1290058,single nucleotide variant,NM_001270974.2(HYDIN):c.2974G>A (p.Ala992Thr),54768,HYDIN,HGNC:19368,Likely benign,0,-,-1,-,RCV001730459,MedGen:CN517202,not provided,germline,germline,GRCh38,NC_000016.10,16,71027670,71027670,na,na,16q22.2,no assertion criteria provided,2,-,N,-,2,1300007,71027670,C,T
2262904,1290059,single nucleotide variant,NM_001324144.2(ZNF41):c.824A>T (p.Glu275Val),7592,ZNF41,HGNC:13107,Likely benign,0,-,-1,-,RCV001730460,MedGen:CN517202,not provided,germline,germline,GRCh38,NC_000023.11,X,47448946,47448946,na,na,Xp11.3,no assertion criteria provided,2,-,N,-,2,1300008,47448946,T,A
2262906,1290060,single nucleotide variant,NM_015335.5(MED13L):c.1654A>G (p.Ile552Val),23389,MED13L,HGNC:22962,Likely benign,0,-,-1,-,RCV001730461,MedGen:CN517202,not provided,germline,germline,GRCh38,NC_000012.12,12,116008759,116008759,na,na,12q24.21,no assertion criteria provided,2,-,N,-,2,1300009,116008759,T,C
2262908,1290061,Microsatellite,NM_001348716.2(KDM6B):c.753ACC[12] (p.Pro264del),23135,KDM6B,HGNC:29012,Benign/Likely benign,0,-,-1,-,RCV001730462|RCV001730463,MedGen:CN517202|MedGen:CN169374,not provided|not specified,germline,germline,GRCh38,NC_000017.11,17,7846860,7846862,na,na,17p13.1,no assertion criteria provided,2,-,N,-,2,1300010,784

In [ ]:
df_variants_summary = df_variants_summary[['Chromosome','Start','ReferenceAlleleVCF','AlternateAlleleVCF','ClinicalSignificance']]
df_variants_summary

,Chromosome,Start,ReferenceAlleleVCF,AlternateAlleleVCF,ClinicalSignificance
1,7,4781213,GGAT,TGCTGTAAACTGTAACTGTAAA,Pathogenic
3,7,4787730,GCTGCTGGACCTGCC,G,Pathogenic
5,15,84799209,G,A,Uncertain significance
7,11,126275389,C,T,Pathogenic
9,11,126277517,A,G,Pathogenic
...,...,...,...,...,...
2262902,16,71027670,C,T,Likely benign
2262904,X,47448946,T,A,Likely benign
2262906,12,116008759,T,C,Likely benign
2262908,17,7846860,TACC,T,Benign/Likely benign


In [ ]:
df_variants_summary.columns = ['chr','position','ref','alt','pathogenicity']

In [ ]:
df_after_merge_annotate_summary = pd.merge(df_after_merge, df_variants_summary, how='left' )
df_after_merge_annotate_summary

,chr,position,gene_names,ref,alt,alt_list,var_type,mol_consequence,biallelic,pathogenicity
0,1,6425219,ESPN,G,A,NaN,single_nucleotide_variant,nonsense,yes,Pathogenic
1,1,6425247,ESPN,C,T,NaN,single_nucleotide_variant,nonsense,yes,Likely pathogenic
2,1,6445936,ESPN,G,A,NaN,single_nucleotide_variant,splice_donor_variant,yes,Likely pathogenic
3,1,6448931,ESPN,C,CG,NaN,Duplication,frameshift_variant,yes,Pathogenic
4,1,6451602,ESPN,G,C,NaN,single_nucleotide_variant,splice_acceptor_variant,yes,Pathogenic
...,...,...,...,...,...,...,...,...,...,...
5325,MT,15927,MT-TT,G,A,NaN,single_nucleotide_variant,NaN,yes,Benign
5326,MT,15975,MT-TP,C,T,NaN,single_nucleotide_variant,NaN,yes,NaN
5327,MT,15992,MT-TP,A,G,NaN,single_nucleotide_variant,NaN,yes,NaN
5328,MT,15997,MT-TP,T,C,NaN,single_nucleotide_variant,NaN,yes,NaN


In [ ]:
df_after_merge_annotate_summary[~df_after_merge_annotate_summary['pathogenicity'].isnull()]

,chr,position,gene_names,ref,alt,alt_list,var_type,mol_consequence,biallelic,pathogenicity
0,1,6425219,ESPN,G,A,NaN,single_nucleotide_variant,nonsense,yes,Pathogenic
1,1,6425247,ESPN,C,T,NaN,single_nucleotide_variant,nonsense,yes,Likely pathogenic
2,1,6445936,ESPN,G,A,NaN,single_nucleotide_variant,splice_donor_variant,yes,Likely pathogenic
3,1,6448931,ESPN,C,CG,NaN,Duplication,frameshift_variant,yes,Pathogenic
4,1,6451602,ESPN,G,C,NaN,single_nucleotide_variant,splice_acceptor_variant,yes,Pathogenic
...,...,...,...,...,...,...,...,...,...,...
5319,MT,14693,MT-TE,A,G,NaN,single_nucleotide_variant,NaN,yes,Benign
5321,MT,15902,MT-TT,A,G,NaN,single_nucleotide_variant,NaN,yes,Benign
5322,MT,15908,MT-TT,T,C,NaN,single_nucleotide_variant,NaN,yes,Benign
5324,MT,15926,MT-TT,C,T,NaN,single_nucleotide_variant,NaN,yes,Benign


In [ ]:
df_after_merge_annotate_summary.to_csv('/content/drive/MyDrive/GRAR/Paper_Surdez/Variants_after_merge_annotated.txt', index = None, header = True, sep = '\t')

### Detecting position on genes....


In [ ]:
df_lost_pos = pd.read_table('/content/drive/MyDrive/GRAR/Paper_Surdez/regioes_corrigido.txt', usecols=['chr','start', 'stop'])
#df_lost_pos['end'] = df_lost_pos['position'].astype(int) +1
df_lost_pos.columns = ['Chromosome', 'Start', 'End']
df_lost_pos['Chromosome'] = df_lost_pos['Chromosome'].str.replace('chr','')
df_lost_pos = df_lost_pos.sort_values(by=['Chromosome', 'Start'])
df_lost_pos = df_lost_pos.drop_duplicates()
df_lost_pos

,Chromosome,Start,End
2009,1,789414,224014574
2018,1,789416,224014443
2027,1,789426,224014501
2036,1,789430,224014567
2045,1,789437,224014519
...,...,...,...
2008,X,129526048,130517124
113990,X,130148900,130148900
113991,X,130148915,130148922
113995,X,130148915,130148916


In [ ]:
transcritos_pr = pr.PyRanges(df_transcritos_merged)
df_lost_pos_pr = pr.PyRanges(df_lost_pos)

transcritos_pr

,Gene,Chromosome,Start,End
0,ESPN,1,6424955,6460146
1,GJB3,1,34784762,34785575
2,KCNQ4,1,40784093,40838523
3,BSND,1,54999186,55008628
4,ROR1,1,63774417,64178855
...,...,...,...,...
121,POU3F4,X,83508324,83509410
122,GPRASP2,X,102714869,102717386
123,PRPS1,X,107628628,107650032
124,COL4A6,X,108156999,108439371


In [ ]:
annotate_transcript = df_lost_pos_pr.join(transcritos_pr).drop(like="_b")
df_lost_pos_annot = annotate_transcript.df

In [ ]:
#df_lost_pos_annot.groupby(['Chromosome','Start','End',])['Gene'].transform(lambda x: ','.join(x))
#df_lost_pos_annot.groupby(['Chromosome','Start','End',])['Gene'].transform(lambda x: ','.join(x)).reset_index()
df_lost_pos_annot.groupby(['Chromosome','Start','End',], as_index = False).agg({'Gene': ', '.join})
#la.to_csv('/content/drive/MyDrive/GRAR/Paper_Surdez/All_positions_annottated.txt', index = None, header = True, sep = '\t')